# FAISS intro 

Reproduction of the code in [FAISS tutorial](https://www.pinecone.io/learn/faiss-tutorial/)

In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

### Take the data

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [3]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [4]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [5]:
# getting more data

urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [6]:
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data_1 = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    # add to columns 1 and 2 to sentences list
    sentences.extend(data_1[1].tolist())
    sentences.extend(data_1[2].tolist())

In [7]:
len(set(sentences))

14505

In [8]:
# removing nan and duplicated
sentences = pd.Series([word for word in list(set(sentences)) if type(word) is str])
len(sentences)

14504

In [9]:
# Creating a BERT model using sentence transformers

from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences.to_list())
sentence_embeddings.shape

(14504, 768)

## Plan and simple similarity search using IndexFlatL2

In [10]:
import faiss

In [12]:
# get the vector size 
d = sentence_embeddings.shape[1]
d

768

In [17]:
# generating the index
index = faiss.IndexFlatIP(d)

In [18]:
# is the index trained?
index.is_trained

True

In [19]:
# addeding our embeddings (vectorized text)
index.add(sentence_embeddings)
# see the total
index.ntotal

14504

In [20]:
# searching similarities the k nearest neigbors between xq and our data 
k = 4
xq = model.encode(["Someone sprints with a football"])

In [21]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[13300 12426 10782  2979]]
CPU times: user 16.7 ms, sys: 1.87 ms, total: 18.6 ms
Wall time: 15.8 ms


In [17]:
# verify our text
sentences.iloc[I[0]]

5848    A group of football players is running in the ...
8755    A group of people playing football is running ...
7368            Two groups of people are playing football
160     A person playing football is running past an o...
dtype: object

In [18]:
# extracting the numerical vector from faiss
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [19]:
vecs.shape

(4, 768)

In [20]:
vecs[0][:100]

array([ 0.01627034,  0.22325909, -0.15037404, -0.30747247, -0.27122435,
       -0.10593174, -0.06460953,  0.04738246, -0.73349065, -0.37657726,
       -0.76762778,  0.16902889,  0.53107685,  0.51176643,  1.14415836,
       -0.08562846, -0.67240065, -0.96637088,  0.02545462, -0.2155983 ,
       -1.25656593, -0.82982177, -0.09825007, -0.21850885,  0.50610268,
        0.10527948,  0.50396878,  0.65242952, -1.39458692,  0.65847504,
       -0.21525346, -0.22487436,  0.81818366,  0.08464289, -0.76141709,
       -0.28928319, -0.09825802, -0.7304616 ,  0.07855845, -0.84354568,
       -0.59242058,  0.77471358, -1.20920539, -0.22757953, -1.30733609,
       -0.23081474, -1.31322527,  0.01629097, -0.97285479,  0.19308148,
        0.47424552,  1.18920887, -1.96741259, -0.70061088, -0.29638764,
        0.60533708,  0.62407476, -0.70340389, -0.86754155,  0.17673104,
       -0.19170557, -0.02951936,  0.22623543, -0.1669542 , -0.80402559,
       -0.45918953,  0.69675452, -0.24928206, -1.01478684, -0.92

## Partitioning the index (IVFFlat)

In [ ]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [ ]:
index.is_trained

In [ ]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

In [ ]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)
# FlatL2 result [[ 9884   340 12395  5399]]

In [ ]:
# verify our text
sentences.iloc[I[0]]

In [ ]:
# increasin the nprobe 
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

In [ ]:
# for the vector reconstruction we need to add direct map
index.make_direct_map()

In [ ]:
index.reconstruct(7460)[:100]

## Quantization

it works in this way
1. We split the original vector into several subvectors.
2. For each set of subvectors, we perform a clustering operation — creating multiple centroids for each sub-vector set.
3. In our vector of sub-vectors, we replace each sub-vector with the ID of it’s nearest set-specific centroid.

In [ ]:
# the worflow
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [ ]:
index.is_trained

In [ ]:
# training the index
index.train(sentence_embeddings)

In [ ]:
# adding our vectors
index.add(sentence_embeddings)

In [ ]:
# setting the nbrobe
index.nprobe = 10  # align to previous IndexIVFFlat 

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

# FlatL2 result [[ 9884   340 12395  5399]] (15.4 ms) 
# IVFFlat result nbrobe = 10 [[ 9884   340 12395  5399]] (4.6 ms)

# FAISS selecting right index

Index selection will depends of 
1. Dataset size
2. Search Frequency
3. Search-Quality vs search-speed

## Flat index

Is called flat because the vectors are not modified

Used
* Search quality is a very high priority.
* Search time does not matter OR when using a small index (<10K).

(See the FlatL2 example above)

To improve velocity we need

1. Reduce vector size — through dimensionality reduction or reducing the number of bits representing our vectors values.

2. Reduce search scope — we can do this by clustering or organizing vectors into tree structures based on certain attributes, similarity, or distance — and restricting our search to closest clusters or filter through most similar branches.

## Locality Sensitive Hassing (LSH)

Depending the parameters used we can obtain a range of efficency quality.

LSH works by grouping vectors into buckets by processing each vector through a hash function that maximizes hashing collisions — rather than minimizing as is usual with hashing functions.

IndexLSH is not suitable if we have large vector dimensionality (128 is already too large). Instead, it is best suited to low-dimensionality vectors — and small indexes.

If we find ourselves with large d values or large indexes — we avoid LSH completely, instead focusing on our next index, HNSW.

In [26]:
# LSH implementation
nbits = d*2  # resolution of bucketed vectors
# initialize index and add vectors
index = faiss.IndexLSH(d, nbits)
index.add(sentence_embeddings)

In [27]:
%%time
D, I = index.search(xq, k)
print(I)
# FlatL2 result [[5848 8755 7368  160]] (15.4 ms) 

[[5848 2862 8755  160]]
CPU times: user 154 ms, sys: 4.7 ms, total: 159 ms
Wall time: 7.2 ms


In [28]:
# FlatL2 result 
sentences.iloc[[5848, 8755, 7368, 160]]

5848    A group of football players is running in the ...
8755    A group of people playing football is running ...
7368            Two groups of people are playing football
160     A person playing football is running past an o...
dtype: object

In [29]:
# verify our text
sentences.iloc[I[0]]

5848    A group of football players is running in the ...
2862    A group of football players running down the f...
8755    A group of people playing football is running ...
160     A person playing football is running past an o...
dtype: object

## Hierarchical Navigable Small World Graphs (HNSW)

HNSW is a further adaption of navigable small world (NSW) graphs — where an NSW graph is a graph structure containing vertices connected by edges to their nearest neighbors.

For bigger datasets with higher-dimensionality — HNSW graphs are some of the best performing indexes we can use.

So, where RAM is not a limiting factor HNSW is great as a well-balanced index that we can push to focus more towards quality by increasing our three parameters.

Some important parameters are:
* ```M``` — the number of nearest neighbors that each vertex will connect to.
* ```efSearch``` — how many entry points will be explored between layers during the search.
* ```efConstruction``` — how many entry points will be explored when building the index.

```M``` and ```efSearch``` have a larger impact on search-time — ```efConstruction``` primarily increases index construction time (meaning a slower index.add), but at higher ```M``` values and higher query volume we do see an impact from ```efConstruction``` on search-time too.

HNSW indexes take up a significant amount of memory.

In [25]:
# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
index = faiss.IndexHNSWFlat(d, M, faiss.METRIC_INNER_PRODUCT)
# set efConstruction and efSearch parameters
index.hnsw.efConstruction = ef_construction
index.hnsw.efSearch = ef_search
# add data to index
index.add(sentence_embeddings)

In [23]:
%%time
D, I = index.search(xq, k)
print(I)
# FlatL2 result [[5848 8755 7368  160]] (15.4 ms) 
# LSH result [[5848 2862 8755  160]] (7.2 ms)

[[13300 12426 10782  2979]]
CPU times: user 22.7 ms, sys: 1.6 ms, total: 24.3 ms
Wall time: 1.74 ms


In [24]:
D

array([[273.63864, 273.54172, 270.8955 , 269.7536 ]], dtype=float32)

In [42]:
# ["Someone sprints with a football"]
sentences.iloc[I[0]]

5848    A group of football players is running in the ...
8755    A group of people playing football is running ...
7368            Two groups of people are playing football
160     A person playing football is running past an o...
dtype: object

##  Inverted File Index

The Inverted File Index (IVF) index consists of search scope reduction through clustering. It’s a very popular index as it’s easy to use, with high search-quality and reasonable search-speed.

Just as with our other indexes, we introduce a query vector xq — this query vector must land within one of our cells, at which point we restrict our search scope to that single cell.

But there is a problem if our query vector lands near the edge of a cell — there’s a good chance that its closest other datapoint is contained within a neighboring cell. We call this the edge problem.

Now, what we can do to mitigate this issue and increase search-quality is increase an index parameter known as the ```nprobe``` value. With ```nprobe``` we can set the number of cells to search.

The parameters:
* ```nprobe``` — the number of cells to search
* ```nlist``` — the number of cells to create

A higher nlist means that we must compare our vector to more centroid vectors — but after selecting the nearest centroid’s cells to search, there will be fewer vectors within each cell. So, increase nlist to prioritize search-speed.

As for nprobe, we find the opposite. Increasing nprobe increases the search scope — thus prioritizing search-quality.

** See IVFFlat example above**

# Final scope...

| Index| Memory (MB) | Query Time (ms) | Recall | Notes
| --- | --- | --- | --- | --- |
| Flat (L2 or IP) | ~500 | ~18 | 1.0 | Good for small datasets or where query time is irrelevant | 
| LSH | 20 - 600 | 1.7 - 30 | 0.4 - 0.85 | Best for low dimensional data, or small datasets |
| HNSW | 600 - 1600 | 0.6 - 2.1 | 0.5 - 0.95 | Very good for quality, high speed, but large memory usage |
| IVF | ~520 | 1 - 9 | 0.7 - 0.95 | Good scalable option. High-quality, at reasonable speed and memory usage |

Values displayed are approximated to a test realized by JamesBriggs [Here](https://www.pinecone.io/learn/vector-indexes/)